In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

# Define paths
train_path = '/kaggle/input/store-sales-time-series-forecasting/train.csv'
test_path = '/kaggle/input/store-sales-time-series-forecasting/test.csv'
stores_path = '/kaggle/input/store-sales-time-series-forecasting/stores.csv'
holidays_path = '/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv'
oil_path = '/kaggle/input/store-sales-time-series-forecasting/oil.csv'
transactions_path = '/kaggle/input/store-sales-time-series-forecasting/transactions.csv'

# Load data
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
stores = pd.read_csv(stores_path)
holidays = pd.read_csv(holidays_path)
oil = pd.read_csv(oil_path)
transactions = pd.read_csv(transactions_path)

# Merge data
train = train.merge(stores, on='store_nbr', how='left')
train = train.merge(holidays, on='date', how='left')
train = train.merge(oil, on='date', how='left')
train = train.merge(transactions, on=['date', 'store_nbr'], how='left')

# Feature engineering
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['dayofweek'] = train['date'].dt.dayofweek

# Encode categorical variables
label_encoder = LabelEncoder()
train['family'] = label_encoder.fit_transform(train['family'])

# Fill missing values
train.fillna(0, inplace=True)

# Define features and target
features = ['store_nbr', 'family', 'onpromotion', 'year', 'month', 'day', 'dayofweek']
X = train[features]
y = train['sales']

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

# Validate model
y_pred = model.predict(X_val)
print('RMSE:', np.sqrt(mean_squared_error(y_val, y_pred)))

# Predict on test data
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['dayofweek'] = test['date'].dt.dayofweek

# Encode categorical variables in test data
test['family'] = label_encoder.transform(test['family'])

test.fillna(0, inplace=True)

X_test = test[features]
test['sales'] = model.predict(X_test)

# Save submission
submission = test[['id', 'sales']]
submission.to_csv('submission.csv', index=False)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.310151 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 354
[LightGBM] [Info] Number of data points in the train set: 2443478, number of used features: 7
[LightGBM] [Info] Start training from score 359.383665
RMSE: 380.78004234563474
